In [ ]:
import singlestoredb as s2
from sentence_transformers import SentenceTransformer
import numpy as np
import time

# ------------------------------
# 1. Connect
# ------------------------------
conn = s2.connect(
    "admin:test@svc-b856f000-4fc6-4d33-83d4-b9b96f90faf0-dml.aws-oregon-4.svc.singlestore.com:3306/amazon10k"
)
print("✅ Connected to SingleStore Helios")

# ------------------------------
# 2. Add embedding column if missing
# ------------------------------
with conn.cursor() as cur:
    try:
        cur.execute("""
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA='amazon'
            AND TABLE_NAME='Reviews_10k'
            AND COLUMN_NAME='embedding_vector'
        """)
        column_exists = cur.fetchone()
        if not column_exists:
            cur.execute("ALTER TABLE Reviews_10k ADD COLUMN embedding_vector BLOB NULL")
            print("✅ embedding_vector column added")
        else:
            print("✅ embedding_vector column already exists")
    except Exception as e:
        print(f"Error with column: {e}")

# ------------------------------
# 3. Load embedding model
# ------------------------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("✅Model loaded")

# ------------------------------
# 4. Generate embeddings
# ------------------------------
with conn.cursor() as cur:
    cur.execute("SELECT id, Text FROM Reviews_10k WHERE embedding_vector IS NULL")
    rows = cur.fetchall()
    print(f"ℹ Processing {len(rows)} rows for embeddings")
    updated_count = 0

    for i, (rid, text_val) in enumerate(rows, 1):
        if text_val and text_val.strip():
            try:
                vec = model.encode(text_val).astype(np.float32).tolist()
                vec_json = str(vec).replace("'", '"')
                cur.execute(
                    "UPDATE Reviews_10k SET embedding_vector = JSON_ARRAY_PACK(%s) WHERE id = %s",
                    (vec_json, rid)
                )
                updated_count += 1
                if i % 100 == 0:
                    print(f"Progress: {i}/{len(rows)} ({i/len(rows)*100:.1f}%)")
            except Exception as e:
                print(f"❌ Error processing ID {rid}: {e}")
        else:
            print(f"⚠️ Skipping ID {rid} - empty text")
    conn.commit()
print(f"✅ Updated {updated_count} embeddings")

# ------------------------------
# 5. Fulltext search
# ------------------------------
def full_text_search(query, topk=5):
    with conn.cursor() as cur:
        # Ensure fulltext index exists
        try:
            cur.execute("""
                SELECT INDEX_NAME
                FROM INFORMATION_SCHEMA.STATISTICS
                WHERE TABLE_SCHEMA='amazon'
                  AND TABLE_NAME='Reviews_10k'
                  AND INDEX_NAME='ft_text'
            """)
            if not cur.fetchone():
                cur.execute("ALTER TABLE Reviews_10k ADD FULLTEXT INDEX ft_text (Text)")
        except:
            pass

        start = time.time()
        cur.execute("""
            SELECT id, Summary, Text,
                   MATCH(Text) AGAINST (%s) AS text_score
            FROM Reviews_10k
            WHERE MATCH(Text) AGAINST (%s)
            ORDER BY text_score DESC
            LIMIT %s
        """, (query, query, topk))
        results = cur.fetchall()
        elapsed = time.time() - start

    print(f"\n--- Full-Text Search Results ({elapsed:.4f}s) ---")
    for i, row in enumerate(results, 1):
        print(f"{i}. ID: {row[0]} | Score: {row[3]:.4f} | Summary: {row[1]}\nText: {row[2][:150]}...\n")
    return results

# ------------------------------
# 6. Vector search
# ------------------------------
def vector_search_dot(query, topk=5):
    query_vec = model.encode(query).astype(np.float32).tolist()
    vec_json = str(query_vec).replace("'", '"')
    with conn.cursor() as cur:
        start = time.time()
        cur.execute("""
            SELECT id, Summary, Text,
                   DOT_PRODUCT(embedding_vector, JSON_ARRAY_PACK(%s)) AS similarity_score
            FROM Reviews_10k
            WHERE embedding_vector IS NOT NULL
            ORDER BY similarity_score DESC
            LIMIT %s
        """, (vec_json, topk))
        results = cur.fetchall()
        elapsed = time.time() - start

    print(f"\n--- Vector Search (Dot Product) Results ({elapsed:.4f}s) ---")
    for i, row in enumerate(results, 1):
        print(f"{i}. ID: {row[0]} | Similarity: {row[3]:.4f} | Summary: {row[1]}\nText: {row[2][:150]}...\n")
    return results

def vector_search_cosine(query, topk=5):
    query_vec = model.encode(query).astype(np.float32).tolist()
    vec_json = str(query_vec).replace("'", '"')

    with conn.cursor() as cur:
        start = time.time()
        cur.execute(f"""
            SELECT id, Summary, Text,
                   (DOT_PRODUCT(embedding_vector, JSON_ARRAY_PACK(%s)) /
                   (SQRT(DOT_PRODUCT(embedding_vector, embedding_vector)) * SQRT(DOT_PRODUCT(JSON_ARRAY_PACK(%s), JSON_ARRAY_PACK(%s))))) AS cosine_similarity
            FROM Reviews_10k
            WHERE embedding_vector IS NOT NULL
            ORDER BY cosine_similarity DESC
            LIMIT %s
        """, (vec_json, vec_json, vec_json, topk))
        results = cur.fetchall()
        elapsed = time.time() - start

    print(f"\n--- Vector Search (Cosine Similarity Fixed) ({elapsed:.4f}s) ---")
    for i, row in enumerate(results, 1):
        print(f"{i}. ID: {row[0]} | Cosine: {row[3]:.4f} | Summary: {row[1]}\nText: {row[2][:150]}...\n")
    return results


# ------------------------------
# 7. Hybrid search (fixed syntax)
# ------------------------------
def hybrid_search(query, topk=5, text_weight=0.3, vector_weight=0.7, strict=False, normalize=False):
    query_vec = model.encode(query).astype(np.float32).tolist()
    vec_json = str(query_vec).replace("'", '"')

    with conn.cursor() as cur:
        # Ensure fulltext index exists
        try:
            cur.execute("ALTER TABLE Reviews_10k ADD FULLTEXT INDEX ft_text (Text)")
        except:
            pass

        start = time.time()

        if normalize:
            # Get max scores for normalization
            cur.execute("""
                WITH stats AS (
                    SELECT
                        MAX(COALESCE(MATCH(Text) AGAINST (%s),0)) AS max_text,
                        MAX(DOT_PRODUCT(embedding_vector, JSON_ARRAY_PACK(%s))) AS max_vector
                    FROM Reviews_10k
                    WHERE embedding_vector IS NOT NULL
                )
                SELECT r.id, r.Summary, r.Text,
                       COALESCE(MATCH(r.Text) AGAINST (%s),0) AS text_score,
                       DOT_PRODUCT(r.embedding_vector, JSON_ARRAY_PACK(%s)) AS vector_score,
                       (CASE WHEN s.max_text>0 THEN %s*(COALESCE(MATCH(r.Text) AGAINST (%s),0)/s.max_text) ELSE 0 END +
                        CASE WHEN s.max_vector>0 THEN %s*(DOT_PRODUCT(r.embedding_vector, JSON_ARRAY_PACK(%s))/s.max_vector) ELSE 0 END) AS hybrid_score
                FROM Reviews_10k r CROSS JOIN stats s
                WHERE r.embedding_vector IS NOT NULL
                """ + (" AND MATCH(r.Text) AGAINST (%s)" if strict else "") + """
                ORDER BY hybrid_score DESC
                LIMIT %s
            """, tuple([query, vec_json, query, vec_json, text_weight, query, vector_weight, vec_json] + ([query] if strict else []) + [topk]))
        else:
            # Raw weighted hybrid score
            cur.execute("""
                SELECT id, Summary, Text,
                       COALESCE(MATCH(Text) AGAINST (%s),0) AS text_score,
                       DOT_PRODUCT(embedding_vector, JSON_ARRAY_PACK(%s)) AS vector_score,
                       (%s*COALESCE(MATCH(Text) AGAINST (%s),0) + %s*DOT_PRODUCT(embedding_vector, JSON_ARRAY_PACK(%s))) AS hybrid_score
                FROM Reviews_10k
                WHERE embedding_vector IS NOT NULL
                """ + (" AND MATCH(Text) AGAINST (%s)" if strict else "") + """
                ORDER BY hybrid_score DESC
                LIMIT %s
            """, tuple([query, vec_json, text_weight, query, vector_weight, vec_json] + ([query] if strict else []) + [topk]))

        results = cur.fetchall()
        elapsed = time.time() - start

    print(f"\n--- Hybrid Search ({'Strict' if strict else 'Relaxed'}, {'Normalized' if normalize else 'Raw'}) ({elapsed:.4f}s) ---")
    for i, row in enumerate(results, 1):
        print(f"{i}. ID: {row[0]} | Hybrid: {row[5]:.4f} (Text: {row[3]:.4f}, Vector: {row[4]:.4f}) | Summary: {row[1]}\nText: {row[2][:150]}...\n")
    return results


# ------------------------------
# 8. Test all search types
# ------------------------------
if __name__ == "__main__":
    query = "good for puppies"

    full_text_search(query)
    vector_search_dot(query)
    vector_search_cosine(query)
    hybrid_search(query, strict=True)
    hybrid_search(query, strict=False)
    hybrid_search(query, strict=False, normalize=True)